In [1]:
import torch
from torch import nn, optim
from torch.utils.data import (Dataset, DataLoader, TensorDataset)
import tqdm

In [2]:
from torchvision.datasets import FashionMNIST
from torchvision import transforms

fashion_mnist_train = FashionMNIST("C:/Users/82109/Desktop/Torch/FashionMNIST",
                                  train = False, download = True, transform = transforms.ToTensor())
fashion_mnist_test = FashionMNIST("C:/Users/82109/Desktop/Torch/FashionMNIST",
                                  train = False, download = True, transform = transforms.ToTensor())

batch_size = 128
train_loader = DataLoader(fashion_mnist_train,
                         batch_size = batch_size, shuffle = True)
test_loader = DataLoader(fashion_mnist_test,
                         batch_size = batch_size, shuffle = False)

Extracting C:/Users/82109/Desktop/Torch/FashionMNIST\FashionMNIST\raw\train-images-idx3-ubyte.gz to C:/Users/82109/Desktop/Torch/FashionMNIST\FashionMNIST\raw


Extracting C:/Users/82109/Desktop/Torch/FashionMNIST\FashionMNIST\raw\train-labels-idx1-ubyte.gz to C:/Users/82109/Desktop/Torch/FashionMNIST\FashionMNIST\raw


Extracting C:/Users/82109/Desktop/Torch/FashionMNIST\FashionMNIST\raw\t10k-images-idx3-ubyte.gz to C:/Users/82109/Desktop/Torch/FashionMNIST\FashionMNIST\raw


Extracting C:/Users/82109/Desktop/Torch/FashionMNIST\FashionMNIST\raw\t10k-labels-idx1-ubyte.gz to C:/Users/82109/Desktop/Torch/FashionMNIST\FashionMNIST\raw
Processing...
Done!


C:\Users\82109\anaconda3\lib\site-packages\torchvision\datasets\mnist.py:469: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [5]:
# (N, C, H, W) 형식의 텐서를 (N, C*H*W)로 늘리는 계층
# 합성곱 출력을 MLP에 전달할 때 필요
class FlattenLayer(nn.Module):
    def forward(self, x):
        sizes = x.size()
        return x.view(sizes[0], -1)
    
conv_net = nn.Sequential(
    nn.Conv2d(1,32,5),
    nn.MaxPool2d(2),
    nn.ReLU(),
    nn.BatchNorm2d(32),
    nn.Dropout2d(0.25),
    nn.Conv2d(32,64,5),
    nn.MaxPool2d(2),
    nn.ReLU(),
    nn.BatchNorm2d(64),
    nn.Dropout2d(0.25),
    FlattenLayer()
    )

In [6]:
# 합성곱에 의해 최종적으로 이미지 크기가 어떤지를 더미 데이터를 넣어 확인
test_input = torch.ones(1,1,28,28)
conv_output_size = conv_net(test_input).size()[-1]

In [10]:
# 2층 MLP

mlp = nn.Sequential(
    nn.Linear(conv_output_size, 200),
    nn.ReLU(),
    nn.BatchNorm1d(200),
    nn.Dropout(0.25),
    nn.Linear(200,10)
)

In [11]:
# 최종 CNN
net = nn.Sequential(
    conv_net,
    mlp
)

In [13]:
# 평가용 헬퍼 함수
def eval_net(net, data_loader, device = "cpu"):
    # Dropout 및 BatchNorm을 무효화
    net.eval()
    ys = []
    ypreds = []
    for x, y in data_loader:
        # to 메서드로 계산을 실행할 디바이스로 전송
        x = x.to(device)
        y = y.to(device)
        # 확률이 가장 큰 분류를 예측
        with torch.no_grad():
            _, y_pred = net(x).max(1)
        ys.append(y)
        ypreds.append(y_pred)
    # 미니 배치 단위의 예측 결과 등을 하나로 묶는다.
    ys = torch.cat(ys)
    ypreds = torch.cat(ypreds)
    # 예측 정확도 계산
    acc = (ys == ypreds).float().sum() / len(ys)
    return acc.item()

In [18]:
# 훈련용 헬퍼 함수
def train_net(net, train_loader, test_loader,
             optimizer_cls = optim.Adam,
             loss_fn = nn.CrossEntropyLoss(),
             n_iter = 10, device = "cpu"):
    train_losses = []
    train_acc = []
    val_acc = []
    optimizer = optimizer_cls(net.parameters())
    for epoch in range(n_iter):
        running_loss = 0.0
        # 신경망을 훈련모드로 설정
        net.train()
        n = 0
        n_acc = 0
        # tqdm을 사용하여 진행바를 표시
        for i, (xx, yy) in tqdm.tqdm(enumerate(train_loader), total = len(train_loader)):
            xx = xx.to(device)
            yy = yy.to(device)
            h = net(xx)
            loss = loss_fn(h, yy)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            n += len(xx)
            _, y_pred = h.max(1)
            n_acc += (yy == y_pred).float().sum().item()
        train_losses.append(running_loss / i)
        train_acc.append(n_acc / n)
        val_acc.append(eval_net(net, test_loader, device))
        # epoch의 결과 표시
        print(epoch, train_losses[-1], train_acc[-1],
             val_acc[-1],flush = True)

In [19]:
# 신경망의 모든 파라미터를 GPU로 전송
net.to("cuda:0")

# 훈련 실행
train_net(net, train_loader, test_loader, n_iter = 20, device = "cuda:0")

100%|██████████████████████████████████████████████████████████████████████████████████| 79/79 [00:02<00:00, 36.13it/s]


0 0.6850146036117505 0.7544 0.8438000082969666


100%|█████████████████████████████████████████████████████████████████████████████████| 79/79 [00:00<00:00, 110.21it/s]


1 0.4708318557494726 0.8306 0.8773999810218811


100%|█████████████████████████████████████████████████████████████████████████████████| 79/79 [00:00<00:00, 110.31it/s]


...

17 0.15618377073835105 0.9421 0.9774999618530273


100%|█████████████████████████████████████████████████████████████████████████████████| 79/79 [00:00<00:00, 109.55it/s]


18 0.14625372904806566 0.9481 0.9764999747276306


100%|█████████████████████████████████████████████████████████████████████████████████| 79/79 [00:00<00:00, 111.40it/s]


19 0.13377916478575805 0.9503 0.9836999773979187


# 타코와 부리토 분류문제

## 1. 데이터 불러오기

In [24]:
from torchvision.datasets import ImageFolder
from torchvision import transforms

# ImageFolder 함수를 사용해서 Dataset 작성
train_imgs = ImageFolder("C:/Users/82109/Desktop/Torch/taco_and_burrito/train/",
                        transform = transforms.Compose([
                            transforms.RandomCrop(224), # 224 * 224 크기로 자른다
                            transforms.ToTensor()]
))

test_imgs = ImageFolder("C:/Users/82109/Desktop/Torch/taco_and_burrito/test/",
                        transform = transforms.Compose([
                            transforms.RandomCrop(224),  
                            transforms.ToTensor()]
))

# DataLoader 작성
train_loader = DataLoader(
    train_imgs, batch_size = 32, shuffle = True)
test_loader = DataLoader(
    test_imgs, batch_size = 32, shuffle = True)

In [29]:
print(train_imgs.classes)
print("train Classes:{}".format(train_imgs.class_to_idx))

['burrito', 'taco']
train Classes:{'burrito': 0, 'taco': 1}


## 2. Pre-trained Model 불러오기

In [38]:
from torchvision import models

# resnet18
net = models.resnet18(pretrained = True)

# 모든 파라미터를 미분 대상에서 제외
for p in net.parameters():
    p.requires_grad = False

# 마지막 선형 계층(softmax class)을 변경
fc_input_dim = net.fc.in_features
net.fc = nn.Linear(fc_input_dim, 2)
print(net.fc)

Linear(in_features=512, out_features=2, bias=True)


## 3. Model의 train function 작성

In [39]:
def eval_net(net, data_loader, device = "cpu"):
    # Dropout 및 BatchNorm을 무효화
    net.eval()
    ys = []
    ypreds = []
    for x, y in data_loader:
        # to 메서드로 계산을 실행할 디바이스로 전송
        x = x.to(device)
        y = y.to(device)
        # 확률이 가장 큰 분류를 예측
        with torch.no_grad():
            _, y_pred = net(x).max(1)
        ys.append(y)
        ypreds.append(y_pred)
    # 미니 배치 단위의 예측 결과 등을 하나로 묶는다.
    ys = torch.cat(ys)
    ypreds = torch.cat(ypreds)
    # 예측 정확도 계산
    acc = (ys == ypreds).float().sum() / len(ys)
    return acc.item()

def train_net(net, train_loader, test_loader, only_fc = True,
             optimizer_cls = optim.Adam,
             loss_fn = nn.CrossEntropyLoss(),
             n_iter = 10, device = "cpu"):
    train_losses = []
    train_acc = []
    val_acc = []
    if only_fc:
        # 마지막 선형계층의 파라미터만 optimizer에 전달
        optimizer = optimizer_cls(net.fc.parameters())
    else:
        optimizer = optimizer_cls(net.parameters())
    for epoch in range(n_iter):
        running_loss = 0.0
        # 신경망을 훈련모드로 설정
        net.train()
        n = 0
        n_acc = 0
        for i, (xx,yy) in tqdm.tqdm(enumerate(train_loader), total = len(train_loader)):
            xx = xx.to(device)
            yy = yy.to(device)
            h = net(xx)
            loss = loss_fn(h,yy)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            n += len(xx)
            _, y_pred = h.max(1)
            n_acc += (yy == y_pred).float().sum().item()
        train_losses.append(running_loss)
        train_acc.append(n_acc / n)
        val_acc.append(eval_net(net,test_loader,device))
        print(epoch, train_losses[-1],train_acc[-1],val_acc[-1],flush = True)

In [40]:
net.to("cuda:0")
train_net(net, train_loader, test_loader, n_iter = 20, device = "cuda:0")

100%|██████████████████████████████████████████████████████████████████████████████████| 23/23 [00:01<00:00, 13.48it/s]

0 16.02640074491501 0.5702247191011236 0.7500000596046448



100%|██████████████████████████████████████████████████████████████████████████████████| 23/23 [00:01<00:00, 13.66it/s]

1 11.921103537082672 0.7668539325842697 0.7666667103767395



100%|██████████████████████████████████████████████████████████████████████████████████| 23/23 [00:01<00:00, 13.74it/s]


...

17 6.6876527070999146 0.8862359550561798 0.8166667222976685


100%|██████████████████████████████████████████████████████████████████████████████████| 23/23 [00:01<00:00, 13.71it/s]

18 6.648392736911774 0.8693820224719101 0.8333333730697632



100%|██████████████████████████████████████████████████████████████████████████████████| 23/23 [00:01<00:00, 13.68it/s]


19 6.482852786779404 0.8820224719101124 0.8333333730697632
